# Retrieval Beyond Similarity: LazyGraphRAG in LangChain

## Introduction

We originally [created the `GraphVectorStore`](https://www.datastax.com/blog/knowledge-graphs-for-rag-without-a-graphdb) as a way of efficiently handling structured relationships between nodes.
We [introduced this to LangChain](https://www.datastax.com/blog/now-in-langchain-graph-vector-store-add-structured-data-to-rag-apps), and [demonstrated significant cost savings](https://hackernoon.com/how-to-save-$70k-building-a-knowledge-graph-for-rag-on-6m-wikipedia-pages) by lazily traversing the metadata instead of eagerly generating the knowledge graph. 
The benefit of lazily traversing nodes based on extracted metadata has since been supported by [Microsoft’s introduction of LazyGraphRAG](https://www.microsoft.com/en-us/research/blog/lazygraphrag-setting-a-new-standard-for-quality-and-cost/).

In this post, we demonstrate two exciting refinements:

1. Replacing the GraphVectorStore with traversing Retrievers. This allows any existing LangChain VectorStore to be used as a graph by lazily traversing metadata fields. This allows an already populated VectorStore to be traversed without making any changes, and really delivers on the promise of lazy GraphRAG – only creating the graph when you perform retrieval.

2. An implementation of LazyGraphRAG performing clustering and summarization on top of any LangChain VectorStore providing metadata filtering.

## Colab Environment Setup

The following block will configure the environment from the Colab Secrets.
To run it, you should have the following Colab Secrets defined and accessible to this notebook:

- `OPENAI_API_KEY`: The OpenAI key.
- `ASTRA_DB_API_ENDPOINT`: The Astra DB API endpoint.
- `ASTRA_DB_APPLICATION_TOKEN`: The Astra DB Application token.
- `LANGCHAIN_API_KEY`: Optional. If defined, will enable LangSmith tracing.
- `ASTRA_DB_KEYSPACE`: Optional. If defined, will specify the Astra DB keyspace. If not defined, will use the default.

In [ ]:
# Install modules.
%pip install -U -r requirements.txt

In [ ]:
# Override LangChain from Eric's fork.
# Won't be needed once this is available.
%pip install --force-reinstall git+https://github.com/epinzur/langchain.git@graph_retrievers#subdirectory=libs/community

In [3]:
# Configure import paths.
import sys
import os
sys.path.append("../../")

# Initialize environment variables.
from utils import initialize_environment, Environment
initialize_environment(Environment.ASTRAPY)

os.environ["LANGCHAIN_PROJECT"] = "2025_LazyGraphRAG"

In [4]:
#@ Create the AstraDBVectorStore

from langchain_openai import OpenAIEmbeddings
from langchain_astradb import AstraDBVectorStore

COLLECTION = "lazy_graph_rag"
store = AstraDBVectorStore(
    embedding=OpenAIEmbeddings(),
    collection_name=COLLECTION,
)

## Part 0: Loading Data

In this section, we demonstrate how to load Wikipedia data into an `AstraDBVectorStore`, using the mentioned articles and entities extracted using [GLiNER](https://github.com/urchade/GLiNER) as fields.
This populates the following metadata fields:

* `id`: Containing the ID of the given article.
* `mentions`: Containing a list of IDs mentioned in the article (populate from the content of the article).
* `entities`: Containing a list of entities mentioned in the article, computed using the `GLiNEREntityExtractor`.

This only needs to be done once to populate the datastore.
At this point, we're just setting metadata on the documents -- in the next part we'll see how to traverse these metadata fields during retrieval.

In [ ]:
#@ Creating Documents from WikiPedia articles

from langchain_core.documents import Document
import json
from typing import Iterator
from datasets.wikimultihop.load import aload_2wikimultihop
from langchain_community.document_transformers.gliner_entitiy_extractor import GLiNEREntityExtractor

def parse_document(line: str) -> Document:
    """Reads one JSON line from the wikimultihop dump."""
    para = json.loads(line)

    id = para["id"]

    # Use structured information (mentioned Wikipedia IDs) as metadata.
    mentioned_ids = [id for m in para["mentions"] for m in m["ref_ids"] or []]

    return Document(
        id = id,
        page_content = " ".join(para["sentences"]),
        metadata = {
            "id": id,
            "mentions": mentioned_ids,
        }
    )

GLINER_TRANSFORMER = GLiNEREntityExtractor(
    labels = ["Person", "Date", "Location", "Event"],
)

# Load data in batches, using GLiNER to extract entities.
def prepare_batch(lines: Iterator[str]) -> Iterator[Document]:
    # Parse documents from the batch of lines.
    docs = [parse_document(line) for line in lines]

    # Apply GLiNER document transformer to extract named entities.
    # This seems to cause index out of bounds errors if the sentence is too long.
    # docs = GLINER_TRANSFORMER.transform_documents(docs)

    return docs

In [ ]:
#@ Loading Data into the Store

# Perform the actual loading. This uses a persistence mechanism to
# allow re-running if the data isn't completely loaded.
#
# This takes awhile.
# On OS X, it is useful to run with `caffeinate -dis` in a shell,
# which prevents the machine from going to sleep and seems to reduce errors.
await aload_2wikimultihop(store, prepare_batch)

At this point, we've created a `VectorStore` with the Wikipedia articles.
Each article is associated with metadata identifying other articles it mentions and entities named in the article.
This can be used for hybrid search -- performing a vector search for articles similar to a specific question *that also mention a specific term*.

In the next section, we'll go a step further and use the mentions and common entities as links between articles, providing a way to navigate and retrieve documents beyond just similarity.

## Part 1: Traversing Retrievers

The traversing retrievers allow any `VectorStore` that supports metadata filtering to be used as a graph.
We just need to declare which metadata fields correspond to edges, and the retriever takes care of the rest.

In our case, we have two kinds of edges:
1. We connect an article to mentioned articles by linking `mentions` to `id`.
2. We connect an article to other articles on the same entity by linking `entities` to `entities`.

### GraphTraversalRetriever

The basic `GraphTraversalRetriever` starts with `start_k` nodes most similar to the query, and then retrieves all edges to a given depth.
It is particularly useful for retrieving a sub-graph around the question.

In [ ]:
from langchain_community.retrievers.graph_traversal import GraphTraversalRetriever, AstraTraversalAdapter

traversal = GraphTraversalRetriever(
    store = AstraTraversalAdapter(store),
    edges = [("mentions", "id"), "entities"],
    # These are optional, with the shown defaults
    start_k = 4,
    depth = 4,
)
traversal.invoke("where is Lithuania?")

### GraphMMRTraversalRetriever
With higher depths, retrieving the entire sub-graph may be equivalent to retrieving the whole graph.
In these cases, the MMR traversal allows retrieving a fixed number of nodes using an MMR scoring algorithm to choose which nodes to traverse while balancing between similarity and diversity.

In [ ]:
from langchain_community.retrievers.graph_mmr_traversal import GraphMMRTraversalRetriever, AstraMMRTraversalAdapter
mmr_traversal = GraphMMRTraversalRetriever(
    store = AstraMMRTraversalAdapter(store),
    edges = [("mentions", "id"), "entities"],
    # These are optional, with the shown defaults
    k = 4,
    depth = 2,
    fetch_k = 100,
    adjacent_k = 10,
    lambda_mult = 0.5,
    score_threshold = float("-inf"),
)
mmr_traversal.invoke("where is Lithuania?")

One nice thing about the links being defined on top of metadata fields is that the same content can be treated as a graph in different ways depending on the situation.
For example, in some cases you may only want to follow the explicit `mentions` to `id` links, and you can do that by just not including the `entities` links.

## Part 2: Lazy Graph RAG via Hierarchical Summarization

As we've noted before, eagerly building a knowledge graph is prohibitively expensive.
Microsoft seems to agree, and recently introduced LazyGraphRAG, which enables GraphRAG to be performed late -- after a query is retrieved.

We implement the LazyGraphRAG technique using the traversing retrievers as follows:

1. Retrieve a good number of nodes using a traversing retrieval.
2. Identify communities in the retrieved sub-graph.
3. Extract claims from each community relevant to the query using an LLM.
4. Rank each of the claims based on the relevance to the question and select the top claims.
5. Generate an answer to the question based on the extracted claims.

### LangChain for Extracting Claims

In [28]:
from typing import Iterable, List, TypedDict
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.runnables import chain
from operator import itemgetter

class Claim(BaseModel):
    """Representation of an individual claim from a source document(s)."""
    claim: str = Field(description="The claim from the original document(s).")
    source_id: str = Field(description="Document ID containing the claim.")


class Claims(BaseModel):
    """Claims extracted from a set of source document(s)."""
    claims: List[Claim] = Field(description="The extracted claims.")

MODEL = ChatOpenAI(model="gpt-4o", temperature=0)
CLAIMS_MODEL = MODEL.with_structured_output(Claims)

CLAIMS_PROMPT = ChatPromptTemplate.from_template("""
Extract claims from the following related documents.

Only return claims appearing within the specified documents.
If no documents are provided, do not make up claims or documents.

Claims (and scores) should be relevant to the question.
Don't include claims from the documents if they are not directly or indirectly relevant to the question.

If none of the documents make any claims relevant to the question, return an empty list of claims.

If multiple documents make similar claims, include the original text of each as separate claims.
Score the most useful and authoritative claim higher than similar, lower-quality claims.

Question: {question}

{formatted_documents}
""")

# TODO: Few-shot examples? Possibly with a selector?

def format_documents_with_ids(documents: Iterable[Document]) -> str:
    formatted_docs = "\n\n".join(
        f"Document ID: {doc.id}\nContent: {doc.page_content}"
        for doc in documents
    )
    return formatted_docs

CLAIM_CHAIN = (
    RunnableParallel({
        "question": itemgetter("question"),
        "formatted_documents": itemgetter("documents") | RunnableLambda(format_documents_with_ids),
    })
   | CLAIMS_PROMPT
   | CLAIMS_MODEL
)

class ClaimsChainInput(TypedDict):
    question: str
    communities: Iterable[Iterable[Document]]

@chain
async def claims_chain(input: ClaimsChainInput) -> Iterable[Claim]:
    question = input["question"]
    communities = input["communities"]

    # TODO: Use openai directly so this can use the batch API for performance/cost?
    community_claims = await CLAIM_CHAIN.abatch([
        { "question": question, "documents": community } for community in communities
    ])
    return [claim for community in community_claims for claim in community.claims]

### LangChain for Ranking Claims

This is based on ideas from [RankRAG](https://arxiv.org/abs/2407.02485).
Specifically, the prompt is constructed so that the next token should be `True` if the content is relevant and `False` if not.
The probability of the token is used to determine the relevance -- `True` with a higher probability is more relevant than `True` with a lesser probability.

In [29]:
import math
from langchain_core.runnables import chain

RANK_PROMPT = ChatPromptTemplate.from_template("""
Rank the relevance of the following claim to the question.
Output "True" if the claim is relevant and "False" if it is not.
Only output True or False.

Question: Where is Seattle?

Claim: Seattle is in Washington State.

Relevant: True

Question: Where is LA?

Claim: New York City is in New York State.

Relevant: False

Question: {question}

Claim: {claim}

Relevant:
""")

def compute_rank(msg):
    logprob = msg.response_metadata["logprobs"]["content"][0]
    prob = math.exp(logprob["logprob"])
    token = logprob["token"]
    if token == "True":
        return prob
    elif token == "False":
        return 1.0 - prob
    else:
        raise ValueError(f"Unexpected logprob: {logprob}")


RANK_CHAIN = (
    RANK_PROMPT
    | MODEL.bind(logprobs=True)
    | RunnableLambda(compute_rank)
)

class RankChainInput(TypedDict):
    question: str
    claims: Iterable[Claim]

@chain
async def rank_chain(input: RankChainInput) -> Iterable[Claim]:
    # TODO: Use openai directly so this can use the batch API for performance/cost?
    claims = input["claims"]
    ranks = await RANK_CHAIN.abatch([
        { "question": input["question"], "claim": claim } for claim in claims
    ])
    rank_claims = sorted(zip(ranks, claims, strict=True),
                         key=lambda rank_claim: rank_claim[0])

    return [claim for _, claim in rank_claims]

## LazyGraphRAG in LangChain

In [30]:
from typing import Optional, Tuple
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import chain
from lazy_graph_rag import create_graph, group_by_community
from langchain_core.language_models import BaseLanguageModel

@chain
async def lazy_graph_rag(question: str,
                         *,
                         retriever: BaseRetriever,
                         edges: Optional[Iterable[str | Tuple[str, str]]] = None,
                         model: BaseLanguageModel,
                         max_tokens: int = 1000) -> str:
    """Retrieve claims relating to the question using LazyGraphRAG.

    Returns the top claims up to the given `max_tokens` as a markdown list.
    """

    if edges is None:
        try:
            edges = retriever.edges
        except AttributeError as _:
            raise ValueError(f"Must specify 'edges' or provide a retriever with 'edges' field defined")

    # 1. Retrieve documents using the (traversing) retriever.
    documents = await retriever.ainvoke(question)

    # 2. Create a graph and extract communities.
    documents_by_id, doc_graph = create_graph(
        documents,
        edges = edges,
        directed = False,
    )
    communities = group_by_community(documents_by_id, doc_graph)

    # 3. Extract claims from the communities.
    claims = await claims_chain.ainvoke({"question": question, "communities": communities})

    # 4. Rank the claims and select claims up to the given token limit.
    result_claims = []
    tokens = 0

    for claim in await rank_chain.ainvoke({"question": question, "claims": claims}):
        claim_str = f"- {claim.claim} (Source: {claim.source_id})"

        tokens += model.get_num_tokens(claim_str)
        if tokens > max_tokens:
            break
        result_claims.append(claim_str)

    return "\n".join(result_claims)

## Using LazyGraphRAG in LangChain

In [31]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.retrievers.graph_mmr_traversal import GraphMMRTraversalRetriever, AstraMMRTraversalAdapter
from langchain_community.retrievers.graph_traversal import GraphTraversalRetriever, AstraTraversalAdapter

EDGES = [("mentions", "id"), "entities"]

RETRIEVER = GraphTraversalRetriever(
    store = AstraTraversalAdapter(store),
    edges = EDGES,
    start_k = 100,
    depth = 3,
)

# RETRIEVER = GraphMMRTraversalRetriever(
#     store = AstraMMRTraversalAdapter(store),
#     edges = EDGES,
#     k = 100,
#     depth = 5,
#     fetch_k = 100,
#     adjacent_k = 25,
#     lambda_mult = 0.8,
#     score_threshold = float("-inf"),
# )

ANSWER_PROMPT = PromptTemplate.from_template("""
Answer the question based on the supporting claims.

Only use information from the claims. Do not guess or make up any information.

Where possible, reference and quote the supporting claims.

Question: {question}

Claims:
{claims}
""")

LAZY_GRAPH_RAG_CHAIN = (
    {  "question": RunnablePassthrough(),
       "claims": RunnablePassthrough() |
           lazy_graph_rag.bind(
               retriever=RETRIEVER,
               model=MODEL,
               max_tokens=1000,
           )
    }
    | ANSWER_PROMPT
    | MODEL
)

In [32]:
await LAZY_GRAPH_RAG_CHAIN.ainvoke("Where is Azerbaijan?")

Claims: ['- Azerbaijani statements assert that the current Armenian territory was under the rule of various Turkic tribes, empires, and khanates until the Treaty of Turkmenchay after the Russo-Persian War of 1826–1828. (Source: 29018074)', '- The government of Azerbaijan claims that the territory of the modern Armenian republic were lands that once belonged to Azerbaijanis. (Source: 29018074)', '- Azerbaijan has many islands along the coast of the Caspian Sea. (Source: 29321112)', '- Azerbaijan is associated with the Nagorno-Karabakh region. (Source: 12725998)', "- Ilham Aliyev, the current president of Azerbaijan, has stated that the territory of Armenia is a part of 'ancient Turk and Azerbaijani land.' (Source: 29018074)", '- Azerbaijan is nearly surrounded by mountains, with the Greater Caucasus range to the northeast along the border with Russia, the Lesser Caucasus range to the west along the border with Armenia, and the Talysh Mountains to the south along the border with Iran. (S

AIMessage(content='Azerbaijan is a country located in the Caucasus region of Eurasia. It is bordered by the Caspian Sea to the east, Georgia and Russia to the north, Iran to the south, and Armenia to the southwest and west. Additionally, Azerbaijan has a short international border with Turkey at the southeastern tip of the Iğdır Province on the Turkish side and at the northwestern tip of the Nakhchivan Autonomous Republic on the Azerbaijani side. Baku is the capital city of Azerbaijan. (Sources: 16278429, 1082, 54722186, 54156046)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 1023, 'total_tokens': 1149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': N

The LazyGraphRAG chain is great when a question needs to consider a large amount of relevant information in order to produce a thorough answer.

## Conclusion

This post introduced _traversing retrievers_ which allow any `VectorStore` to be traversed as a knowledge graph based on properties in the metadata.
This means you can focus on populating and using your `VectorStore` with useful metadata and add GraphRAG when you need it.
We also saw that these traversing retrievers mean that any `VectorStore` can be used with LazyGraphRAG, without needing to change the stored documents.

Knowledge Graphs and GraphRAG shouldn't be hard or scary.
Start simple and easily overlay edges when you need them.

These traversing retrievers and LazyGraphRAG summarization work well with agents.
You can create tools that use different retriever configurations, for instance, searching for articles "near" existing articles or distinguishing between questions that only need a few references and deeper questions which need to retrieve and summarize a larger amount of content.
We'll show how to combine these graph techniques with agents in future posts.
Until then, ...